In [79]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from webdriver_manager.chrome import ChromeDriverManager

from bs4 import BeautifulSoup as BS
from datetime import datetime

import pandas as pd
import time
import sys, os
import re
import json


def find_css(css_selector, browser):
    return browser.find_element(By.CSS_SELECTOR, css_selector)
def finds_css(css_selector, browser):
    return browser.find_elements(By.CSS_SELECTOR, css_selector)

def find_xpath(xpath, browser):
    return browser.find_element(By.XPATH, xpath)
def finds_xpath(xpath, browser):
    return browser.find_elements(By.XPATH, xpath)

def find_id(e_id, browser):
    return browser.find_element(By.ID, e_id)

def find_className(cn, browser):
    return browser.find_element(By.CLASS_NAME, cn)
def finds_className(cn , browser):
    return browser.find_elements(By.CLASS_NAME, cn)

def find_linktext(lt, browser):
    return browser.find_element(By.LINK_TEXT, lt)

def find_name(name, browser):
    return browser.find_element(By.NAME, name)
def finds_name(name, browser):
    return browser.find_elements(By.NAME, name)

def find_tagName(tag_name, browser):
    return browser.find_element(By.TAG_NAME, tag_name)

def finds_tagName(tag_name, browser):
    return browser.find_elements(By.TAG_NAME, tag_name)

def resource_path(relative_path):
    """ Get absolute path to resource, works for dev and for PyInstaller """
    base_path = getattr(sys, '_MEIPASS', os.path.dirname(os.path.abspath(__file__)))
    return os.path.join(base_path, relative_path)

In [80]:
def open_browser():
    options = webdriver.ChromeOptions()
    options.add_argument('--no--sandbox')
    options.add_argument('no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--window-size=1080,800')
    options.add_argument('incognito')
    chrome_service = Service('chromedriver')
    chrome_service = Service(executable_path="chromedriver.exe")
    browser = webdriver.Chrome(service=chrome_service, options=options)
    
    return browser

In [81]:
def rm_login(rm_id, rm_pw, browser):
    browser.get("https://www.rm1.co.kr/")
    
    try:
        browser.switch_to.frame(0)
    except Exception as ex:
        print(ex)
        pass
    
    time.sleep(.5)
    find_id("stay_sign", browser).click()
    time.sleep(.5)
    try:
        alert = browser.switch_to.alert
        alert.accept()
    except Exception as ex:
        print(ex)
        pass
    
    find_id('user_id', browser).send_keys(rm_id)
    time.sleep(.5)
    find_id('user_pw', browser).send_keys(rm_pw)
    time.sleep(.5)
    find_id('user_pw', browser).send_keys("\n")
    
    time.sleep(2)
    
    try:
        find_id('closeToday', browser).click()
        time.sleep(.3)
        find_id('closeBtn', browser).click()
    except:
        pass

In [82]:
def first_search_company(business_number, browser):
    try:
        search_input = find_id('searchTxt', browser)
        search_input.clear()
        search_input.send_keys(business_number)
        time.sleep(1)
        search_input.send_keys('\n')

        time.sleep(1)
    except:
        browser.get("https://www.rm1.co.kr/")
        try:
            browser.switch_to.frame(0)
        except Exception as ex:
            print(ex)
            pass
        
        time.sleep(1)
        
        search_input = find_id('searchTxt', browser)
        search_input.clear()
        search_input.send_keys(business_number)
        time.sleep(1)
        search_input.send_keys('\n')

        time.sleep(1)

In [83]:
def extract_company_info(browser):
    # 첫번째 나온 회사 링크 클릭
    first_element = find_id('schResultCmpName', browser)
    find_tagName('a', first_element).click()

    time.sleep(3)
    
    soup = BS(browser.page_source, "html.parser")
    soup = soup.find(class_="tbl_l_a mt5")

    company_data = {}

    keys_to_extract = ['기업명', '대표자명', '사업자번호', '종업원수']

    for row in soup.select('table tbody tr'):
        th = row.find('th', scope='row')
        td = row.find('td').get_text(strip=True) if row.find('td') else None
        if th:
            key = th.get_text(strip=True)
            if key in keys_to_extract:
                company_data[key] = td

#     json_data = json.dumps(company_data, ensure_ascii=False, indent=4)
    
    return company_data

In [84]:
def extract_financial_statements(browser, company_data):
    # 재무상태표 click
    find_id('fsdtset1', browser).click()
    
    time.sleep(1)

    financial_statements = {}
    financial_statements_name = []

    soup = BS(browser.page_source, "html.parser")
    table = soup.find(id='fin_a')
    header = table.find('thead').find_all('th')[1:]

    for th in header:
        date = th.get_text(strip=True)
        financial_statements[date] = []

    rows = table.find('tbody').find_all('tr')
    for row in rows:
        columns = row.find_all('td')
        for i, column in enumerate(columns):
            date = list(financial_statements.keys())[i]
            value = column.get_text(strip=True)
            financial_statements[date].append(value)
    
    try:
        financial_statements['2022-12-31(결산)'].insert(0, company_data['기업명'])
        financial_statements['2022-12-31(결산)'].insert(0, company_data['사업자번호'])
        financial_statements_name.insert(0, '회사이름')
        financial_statements_name.insert(1, '사업자번호')
    except:
        pass
    
    if '금액' in financial_statements:
        del financial_statements['금액']

    # print(json.dumps(financial_statements, ensure_ascii=False, indent=4))

    resoup = soup.find(class_="td_dp01")
    th_col_name = resoup.find_all('th')
    for col in th_col_name:
        financial_statements_name.append(col.text)

    return financial_statements_name, financial_statements

In [85]:
def extract_income_statement(browser, company_data):
    find_id('li12', browser).click()
    
    income_statement_value = {}
    income_statement_col_name = []

    soup = BS(browser.page_source, "html.parser")
    table = soup.find(id='fin_a')
    header = table.find('thead').find_all('th')[1:]

    for th in header:
        date = th.get_text(strip=True)
        income_statement_value[date] = []

    rows = table.find('tbody').find_all('tr')
    for row in rows:
        columns = row.find_all('td')
        for i, column in enumerate(columns):
            date = list(income_statement_value.keys())[i]
            value = column.get_text(strip=True)
            income_statement_value[date].append(value)
    
    try:
        income_statement_value['2022-12-31(결산)'].insert(0, company_data['기업명'])
        income_statement_value['2022-12-31(결산)'].insert(1, company_data['사업자번호'])
        income_statement_col_name.insert(0, '회사이름')
        income_statement_col_name.insert(1, '사업자번호')
    except:
        pass
    
    if '금액' in income_statement_value:
        del income_statement_value['금액']

    # print(json.dumps(financial_statements, ensure_ascii=False, indent=4))

    resoup = soup.find(class_="td_dp01")
    th_col_name = resoup.find_all('th')
    for col in th_col_name:
        income_statement_col_name.append(col.text)
    
    return income_statement_col_name, income_statement_value

In [86]:
xls = pd.ExcelFile('회사사업자번호.xlsx')
df = pd.read_excel(xls, '상위')
print(df.shape)
df['사업자번호'] = df['사업자번호'].astype('str')
print("중복치 확인 수 : ",df.duplicated().sum())
df.drop_duplicates(inplace=True)
print(df.shape)

df_1 = pd.read_excel(xls, '전수')
print(df_1.shape)
df_1['사업자번호'] = df_1['사업자번호'].astype('str')
print("중복치 확인 수 : ",df_1.duplicated().sum())
df_1.drop_duplicates(inplace=True)
print(df_1.shape)

df_2 = pd.read_excel(xls, '표본')
print(df_2.shape)
df_2['사업자번호'] = df_2['사업자번호'].astype('str')
print("중복치 확인 수 : ",df_2.duplicated().sum())
df_2.drop_duplicates(inplace=True)
print(df_2.shape)

(220, 1)
중복치 확인 수 :  0
(220, 1)
(2395, 1)
중복치 확인 수 :  0
(2395, 1)
(16575, 1)
중복치 확인 수 :  0
(16575, 1)


In [ ]:
count = 1

final_company_data = []

final_financial_statements_data = []
final_income_statement_data = []

final_financial_statements_col_name = []
final_income_statement_col_name = []

browser = open_browser()

rm_login('rlaxodud3006', 'rlaxodud3006@', browser)

for business_number in df_2['사업자번호']:
    try:
        first_search_company(business_number, browser)

        time.sleep(1.5)
        try:
            company_data = extract_company_info(browser)
            final_company_data.append(company_data)

            financial_statements_name, financial_statements = extract_financial_statements(browser, company_data)

            income_statement_col_name, income_statement_value = extract_income_statement(browser, company_data)
        except Exception:
            pass

        try:
            if '2022-12-31(결산)' in income_statement_value:
                final_income_statement_data.append(income_statement_value['2022-12-31(결산)'])
                final_income_statement_col_name.append(income_statement_col_name)

            if '2022-12-31(결산)' in financial_statements:
                final_financial_statements_data.append(financial_statements['2022-12-31(결산)'])
                final_financial_statements_col_name.append(financial_statements_name)

        except Exception as e:
            print(f"An exception occurred: {str(e)}")
            pass

        print(count)
        count += 1
        
    except NoSuchElementException:
        print("Element not found. Logging in again.")
        rm_login('rlaxodud3006', 'rlaxodud3006@', browser)
        continue

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
Element not found. Logging in again.
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
Element not found. Logging in again.
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
25

In [68]:
# 회사 이름 정보
print(len(final_company_data))

print(len(final_financial_statements_data))
print(len(final_income_statement_data))

print(len(final_financial_statements_col_name))
print(len(final_income_statement_col_name))

2327
2054
2054
2054
2054


In [69]:
df2 = pd.DataFrame(final_financial_statements_data)
df2_1 = pd.DataFrame(final_financial_statements_col_name)
df3 = pd.DataFrame(final_income_statement_data)
df3_1 = pd.DataFrame(final_income_statement_col_name)
df4 = pd.DataFrame(final_company_data)

In [70]:
df2.to_excel('표본\\표본_제무제표1_value.xlsx', index=False)
df2_1.to_excel('표본\\표본_제무제표1_key.xlsx', index=False)
df3.to_excel('표본\\표본_손익계산서1_value.xlsx', index=False)
df3_1.to_excel('표본\\표본_손익계산서1_key.xlsx', index=False)
df4.to_excel('표본\\회사인원정보1_전수.xlsx', index=False)

In [71]:
# 재무제표
list_of_lists = df2.values.tolist()
list_of_lists_col = df2_1.values.tolist()

final_list = []

for i, j in zip(list_of_lists_col, list_of_lists):
    final_list.append(i)
    final_list.append(j)
    
final_df = pd.DataFrame(final_list)

# final_df.to_excel('재무제표1 예시.xlsx', index=False)

In [72]:
# 손익계산서

list_of_lists1 = df3.values.tolist()
list_of_lists_col1 = df3_1.values.tolist()

final_list1 = []

for i, j in zip(list_of_lists_col1, list_of_lists1):
    final_list1.append(i)
    final_list1.append(j)
    
final_df1 = pd.DataFrame(final_list1)

# final_df1.to_excel('손익계산서 예시.xlsx', index=False)

In [73]:
print(len(list_of_lists1))
print(len(list_of_lists_col1))

2054
2054


In [74]:
# 재무제표 전처리
final_dict1 = []

for i ,j in zip(list_of_lists_col, list_of_lists):
    result2 = dict(zip(i, j))
    final_dict1.append(result2)

# 손익계산서 전처리
final_dict = []

for i ,j in zip(list_of_lists_col1, list_of_lists1):
    result1 = dict(zip(i, j))
    final_dict.append(result1)

In [75]:
final_lists1 = []

for ai in final_dict:
    try:
        filtered_data = {key: ai[key] for key in selected_items}
        final_lists1.append(filtered_data)
    except:
        pass


column_names = [
    '매출액', '매출원가', '판매비와관리비', '급료와임금', '퇴직급여충당금전입액',
    '복리후생비', '세금과공과', '임차료비용', '감가상각비', '경상연구개발비',
    '대손상각비', '영업이익(손실)', '이자수익', '이자비용'
]

result_dict1 = {}

for company_data in final_dict:
    company_name = company_data['회사이름']
    
    company_result = {column_name: 0 for column_name in column_names}
    
    for column_name in column_names:
        if column_name in company_data:
            value = company_data[column_name]
        else:
            value = 0
        
        company_result[column_name] = value
    
    result_dict1[company_name] = company_result
    
print(len(result_dict1))
print(result_dict1)

2021
{'(주)경동': {'매출액': '128,118,290', '매출원가': '108,923,669', '판매비와관리비': '9,793,072', '급료와임금': '2,613,557', '퇴직급여충당금전입액': '514,111', '복리후생비': '523,650', '세금과공과': '815,756', '임차료비용': '25,161', '감가상각비': '178,849', '경상연구개발비': 0, '대손상각비': '842,306', '영업이익(손실)': '9,401,549', '이자수익': '609,886', '이자비용': '502,215'}, '(주)한강식품': {'매출액': '199,354,725', '매출원가': '193,562,591', '판매비와관리비': '13,319,881', '급료와임금': '3,551,818', '퇴직급여충당금전입액': '291,482', '복리후생비': '556,584', '세금과공과': '153,052', '임차료비용': '276,998', '감가상각비': '1,547,756', '경상연구개발비': 0, '대손상각비': '7,269', '영업이익(손실)': '-7,527,747', '이자수익': '451,552', '이자비용': '3,519,824'}, '(주)하림': {'매출액': '1,289,857,319', '매출원가': '1,100,015,896', '판매비와관리비': '154,178,387', '급료와임금': '33,929,027', '퇴직급여충당금전입액': '2,262,282', '복리후생비': '4,269,596', '세금과공과': '2,582,678', '임차료비용': '1,848,059', '감가상각비': '8,911,316', '경상연구개발비': 0, '대손상각비': '-80,990', '영업이익(손실)': '35,663,037', '이자수익': '3,813,513', '이자비용': '17,887,792'}, '농업회사법인(주)사조원': {'매출액': '338,246,612', '매출원가': '309,19

In [76]:
income_df = pd.DataFrame(result_dict1).T
income_df.shape

income_df.to_excel("표본\\손익계산서_final_전수.xlsx")

In [77]:
column_names = ['유형자산(계)', '토지', '건물', '구축물', '(구축물감가상각누계액)', '기계장치', '(기계장치감가상각누계액)', '차량운반구', '(차량운반구감가상각누계액)', '공구와기구', '(공구와기구감가상각누계액)', '비품', '(비품감가상각누계액)', '기타유형자산', '(기타유형자산감가상각누계액)', '건설중인자산', '무형자산(계)', '(상각누계액)']
result_dict = {}

for company_data in final_dict1:
    company_name = company_data['회사이름']
    
    company_result = {column_name: 0 for column_name in column_names}
    
    for column_name in column_names:
        if column_name in company_data:
            value = company_data[column_name]
        else:
            value = 0
        
        company_result[column_name] = value
    
    result_dict[company_name] = company_result
    
print(len(result_dict))
print(result_dict)

2023
{'222-81-03192': {'유형자산(계)': '40,752,214', '토지': '4,928,182', '건물': '6,265,124', '구축물': '2,960,298', '(구축물감가상각누계액)': '6,803,732', '기계장치': '25,118,467', '(기계장치감가상각누계액)': '135,742,282', '차량운반구': '1,215,640', '(차량운반구감가상각누계액)': '6,124,011', '공구와기구': '37,917', '(공구와기구감가상각누계액)': '877,832', '비품': '117,349', '(비품감가상각누계액)': '2,747,893', '기타유형자산': 0, '(기타유형자산감가상각누계액)': 0, '건설중인자산': 0, '무형자산(계)': '1,520,655', '(상각누계액)': '115,804'}, '127-81-61937': {'유형자산(계)': '218,451,408', '토지': '32,466,027', '건물': '102,447,920', '구축물': '5,385,348', '(구축물감가상각누계액)': '1,769,463', '기계장치': '66,356,774', '(기계장치감가상각누계액)': '14,575,059', '차량운반구': '72,269', '(차량운반구감가상각누계액)': '41,282', '공구와기구': 0, '(공구와기구감가상각누계액)': 0, '비품': '2,427,658', '(비품감가상각누계액)': '2,256,322', '기타유형자산': 0, '(기타유형자산감가상각누계액)': 0, '건설중인자산': '6,659,012', '무형자산(계)': '101,072', '(상각누계액)': 0}, '403-81-61113': {'유형자산(계)': '429,701,376', '토지': '17,151,964', '건물': '165,625,448', '구축물': '8,622,991', '(구축물감가상각누계액)': '10,629,740', '기계장치': '82,081,498', '(기계장치

In [78]:
income_df = pd.DataFrame(result_dict).T
income_df.shape

income_df.to_excel("표본\\재무제표_final_전수.xlsx")